In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
data = pd.read_csv('C:\\Users\\7334\\spotify_ccClub\\country_data\\data_total.csv')

In [4]:
data

,track_name,track_id,popularity,album_name,album_id,release_date,artist_name,artist_id,danceability,energy,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres,country
0,Dakiti,47EiUVwUp4C9fGccaPuUCS,100,Dakiti,43dl8hP5uNN9s6YeeJA5fv,2020-10-30,Bad Bunny,4q3ewBCX7sLwd24euuV69X,0.731,0.573,...,109.928,audio_features,47EiUVwUp4C9fGccaPuUCS,spotify:track:47EiUVwUp4C9fGccaPuUCS,https://api.spotify.com/v1/tracks/47EiUVwUp4C9...,https://api.spotify.com/v1/audio-analysis/47Ei...,205090,4,"['latin', 'reggaeton', 'trap latino']",Global
1,Mood (feat. iann dior),3tjFYV6RSFtuktYl3ZtYcq,99,Mood (feat. iann dior),4YMnOf4a7obOcN1Gy2QEuM,2020-07-24,24kGoldn,6fWVd57NKTalqvmjRd2t8Z,0.700,0.722,...,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,['cali rap'],Global
2,Dynamite,0t1kP63rueHleOhQkYSXFY,96,Dynamite (DayTime Version),6K4chJALBBMYmXjwgvqahx,2020-08-28,BTS,3Nrfpe0tUJi4K4DXYWgMUX,0.746,0.765,...,114.044,audio_features,0t1kP63rueHleOhQkYSXFY,spotify:track:0t1kP63rueHleOhQkYSXFY,https://api.spotify.com/v1/tracks/0t1kP63rueHl...,https://api.spotify.com/v1/audio-analysis/0t1k...,199054,4,"['k-pop', 'k-pop boy group']",Global
3,positions,35mvY5S1H3J2QZyna3TFe0,96,Positions,3euz4vS7ezKGnNSwgyvKcd,2020-10-30,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,0.737,0.802,...,144.015,audio_features,35mvY5S1H3J2QZyna3TFe0,spotify:track:35mvY5S1H3J2QZyna3TFe0,https://api.spotify.com/v1/tracks/35mvY5S1H3J2...,https://api.spotify.com/v1/audio-analysis/35mv...,172325,4,"['pop', 'post-teen pop']",Global
4,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,95,After Hours,4yP0hdKOZPNshxUOjY0cZj,2020-03-20,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.514,0.730,...,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,"['canadian contemporary r&b', 'canadian pop', ...",Global
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180922,Gold,1h7cpRDFr3EOT3Zc7sDqFL,0,Critical As Water,47E2YFEvjYO7xRCtByiVnd,2018-03-16,Jeremy Loops,0Dct2Gu0qEbgGRjfaxew8g,0.688,0.673,...,94.978,audio_features,1h7cpRDFr3EOT3Zc7sDqFL,spotify:track:1h7cpRDFr3EOT3Zc7sDqFL,https://api.spotify.com/v1/tracks/1h7cpRDFr3EO...,https://api.spotify.com/v1/audio-analysis/1h7c...,185000,4,"['cape town indie', 'folk-pop', 'south african...",South Africa
180923,Zone,74XGiFEyklRJ97XpIbO91B,0,Touch My Blood,03FrlITEhM39a7hSZH3Hm9,2018-06-15,AKA,1QIghPIrXQQ22G1yNtAKFX,0.826,0.611,...,95.003,audio_features,74XGiFEyklRJ97XpIbO91B,spotify:track:74XGiFEyklRJ97XpIbO91B,https://api.spotify.com/v1/tracks/74XGiFEyklRJ...,https://api.spotify.com/v1/audio-analysis/74XG...,226920,4,"['afro house', 'afro soul', 'south african alt...",South Africa
180924,March 14,40IiSjljbAhrSbdJ3HxSfH,0,Scorpion,2o9McLtDM7mbODV7yZF2mc,2018-06-29,Drake,3TVXtAsR1Inumwj472S9r4,0.500,0.419,...,91.963,audio_features,40IiSjljbAhrSbdJ3HxSfH,spotify:track:40IiSjljbAhrSbdJ3HxSfH,https://api.spotify.com/v1/tracks/40IiSjljbAhr...,https://api.spotify.com/v1/audio-analysis/40Ii...,309560,4,"['canadian hip hop', 'canadian pop', 'hip hop'...",South Africa
180925,Ellie,2NPiBHafyMwJXvSemJ2S0F,0,Ellie,2Q1TUsalFIRy7mTweIfNxm,2018-05-11,Regi,77ehFS1P2bU6Bfcs1qu6Jd,0.791,0.702,...,124.995,audio_features,2NPiBHafyMwJXvSemJ2S0F,spotify:track:2NPiBHafyMwJXvSemJ2S0F,https://api.spotify.com/v1/tracks/2NPiBHafyMwJ...,https://api.spotify.com/v1/audio-analysis/2NPi...,164929,4,"['belgian dance', 'belgian edm']",South Africa


In [71]:
data_genres = data.copy()[['track_name', 'genres', 'country']]
# 去掉沒有genres的資料, countrty改nation
data_genres = data_genres[~(data_genres['genres'] == '[]')].rename({'country':'nation'}, axis = 'columns')

# 先把 data_genres 做起來, 後續再填資料進去
genres = ['indie', 'blues', 'classical', 'country', 'disco', 'hip hop', 'rap', 'jazz', 'metal', 'pop', 'reggae', 'rock', 'electronic', 'others']
data_genres = pd.concat([data_genres, pd.DataFrame(columns = genres)], axis = 1)

def onehot(lst, g):
    if g in lst:
        return 1
    else:
        return 0
    
# 如果是該曲風則為1, 不是則為0
for gen in genres:
    data_genres[gen] = data_genres['genres'].apply(lambda x: onehot(x, gen))

# 判斷是否為others
boo = data_genres[['indie', 'blues', 'classical', 'country', 'disco', 'hip hop', 'rap', 'jazz', 'metal', 'pop', 'reggae', 'rock', 'electronic']].sum(axis = 1) == 0
data_genres.loc[boo, 'others'] = 1

# 計算各個國家歌曲的總數
nation_counts = data_genres.groupby('nation').count()['track_name']

# 算各曲風總和
nation_genres = data_genres.groupby('nation').sum()

# 加入各國家歌曲總數
nation_genres = pd.concat([nation_genres, nation_counts], axis = 1).rename({'track_name':'counts'}, axis = 'columns')

#合併 hip hop 和 rap
intersect = data_genres[(data_genres['hip hop'] == 1) & (data_genres['rap'] == 1)].groupby('nation').count()['track_name']
hiphop_rap = nation_genres['hip hop'] + nation_genres['rap'] - intersect
#nation_genres.drop(['hip hop', 'rap'], axis = 1, inplace = True)
nation_genres.insert(13, 'hip hop & rap', hiphop_rap)

# 調整 column 順序
nation_genres = nation_genres.reindex(columns = ['indie', 'blues', 'classical', 'country', 'disco', 
                                                 'jazz', 'metal', 'reggae', 'rock', 'electronic', 
                                                 'others', 'hip hop & rap', 'counts', 'pop', 
                                                 'hip hop', 'rap'])
nation_genres


,indie,blues,classical,country,disco,jazz,metal,reggae,rock,electronic,others,hip hop & rap,counts,pop,hip hop,rap
nation,,,,,,,,,,,,,,,,
Andorra,0,0,0,0,0,0,0,51,2,1,10,66,145,96,16,65
Argentina,48,0,0,4,0,0,9,672,166,16,49,1046,2068,1128,481,1031
Australia,221,9,0,28,9,28,24,27,204,13,92,1022,2572,1792,644,939
Austria,163,6,2,16,12,23,43,53,235,19,115,2351,3877,1808,2001,1197
Belgium,148,14,1,19,13,39,46,40,250,20,209,1934,3589,2601,1570,1672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ukraine,18,0,0,2,0,0,12,4,24,7,10,334,537,286,279,221
United Kingdom,166,15,1,18,23,38,44,15,370,12,134,1552,3378,1972,1129,1016
United States,102,8,1,158,3,28,19,102,227,12,98,2422,3985,2537,1658,2351
